**Alzheimer's Detection Project**

In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile

# Define our example directories and files
base_dir = '/content/drive/MyDrive/Alzheimer_s Dataset'

train_dir = os.path.join( base_dir, 'train')
test_dir = os.path.join( base_dir, 'test')

batch_size1 = 32
img_height = 208
img_width = 176
numClasses=4
epoch1=50


train_MildDemented_dir = os.path.join(train_dir, 'MildDemented') 
train_ModerateDemented_dir = os.path.join(train_dir, 'ModerateDemented') 
train_NonDemented_dir = os.path.join(train_dir, 'NonDemented')
train_VeryMildDemented_dir = os.path.join(train_dir, 'VeryMildDemented')

test_MildDemented_dir = os.path.join(test_dir, 'MildDemented') 
test_ModerateDemented_dir = os.path.join(test_dir, 'ModerateDemented') 
test_NonDemented_dir = os.path.join(test_dir, 'NonDemented')
test_VeryMildDemented_dir = os.path.join(test_dir, 'VeryMildDemented')


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1.0/255.0,rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest' )

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255.0 )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = batch_size1,
                                                    class_mode = 'categorical', 
                                                    target_size = (img_height,img_width))     

# Flow validation images in batches of 20 using test_datagen generator
test_generator =  test_datagen.flow_from_directory( test_dir,
                                                          batch_size  = batch_size1,
                                                          class_mode  = 'categorical', 
                                                          target_size = (img_height, img_width))

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


Importing necessary libraries

In [29]:
import tensorflow as tf

In [30]:
import keras
import numpy as np
from keras.applications import vgg16
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

MODEL Building

In [31]:
base_model= keras.applications.vgg16.VGG16(weights= 'imagenet', include_top= False, input_shape=(img_height, img_width, 3))

add_model= Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation= 'relu'))
add_model.add(Dense(256, activation= 'relu'))
add_model.add(Dense(4, activation='softmax'))

model= Model(inputs= base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 208, 176, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 208, 176, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 208, 176, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 104, 88, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 104, 88, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 104, 88, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 52, 44, 128)       0   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [32]:
#loading already trained model for further training
#from keras.models import load_model
#model=load_model('/content/drive/MyDrive/Alzheimer_s Dataset/alzheimers125.h5')

*Training the model*

In [33]:
model.fit_generator(train_generator,
        epochs=25,
        validation_data=test_generator,
        verbose=1,)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/25
161/161 [==============================] - 1061s 6s/step - loss: 0.0711 - accuracy: 0.9742 - val_loss: 1.0845 - val_accuracy: 0.7342
Epoch 2/25
161/161 [==============================] - 77s 480ms/step - loss: 0.0604 - accuracy: 0.9779 - val_loss: 1.1729 - val_accuracy: 0.7482
Epoch 3/25
161/161 [==============================] - 77s 476ms/step - loss: 0.0747 - accuracy: 0.9736 - val_loss: 1.1319 - val_accuracy: 0.7639
Epoch 4/25
161/161 [==============================] - 77s 479ms/step - loss: 0.0546 - accuracy: 0.9826 - val_loss: 1.0980 - val_accuracy: 0.7490
Epoch 5/25
161/161 [==============================] - 77s 477ms/step - loss: 0.0597 - accuracy: 0.9801 - val_loss: 1.0614 - val_accuracy: 0.7482
Epoch 6/25
161/161 [==============================] - 77s 474ms/step - loss: 0.0541 - accuracy: 0.9785 - val_loss: 1.2716 - val_accuracy: 0.7349
Epoch 7/25
161/161 [==============================] - 76s 474ms/step - loss: 0.0562 - accuracy: 0.9801 - val_loss: 1.0774 - val_acc

*Saving the model for further use*

In [34]:
#from keras.models import load_model
model.save("alzheimers.h5")

***#Driver code for user input***

In [35]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()
d={'0':'Mild Dementia','1':'Moderate Dementia','2':'No Dementia','3':'Very Mild Dementia'}

for fn in uploaded.keys():

  path = '/content/' + fn
   
  
  img=image.load_img(path,target_size=(208,176))
  
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=batch_size1)
  key=str(np.argmax(classes[0]))
  print("\nImage uploaded:",fn)
  print("\nPREDICTION:",d[key])

Saving mildDem0.jpg to mildDem0 (9).jpg

Image uploaded: mildDem0.jpg

PREDICTION: Mild Dementia
